<a href="https://colab.research.google.com/github/SalmanShahzadAli/HealthMate---Software-Engineering-Semester-Project/blob/main/Medical_AI_Chatbot_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "header",
   "metadata": {},
   "source": [
    "# Medical AI Chatbot with Llama and RAG\n",
    "## Features:\n",
    "- Uses FREE local Llama model (no API costs!)\n",
    "- Answer basic medical queries using RAG\n",
    "- Appointment scheduling and management\n",
    "- PostgreSQL database integration\n",
    "\n",
    "## Prerequisites:\n",
    "1. Python 3.9 or higher installed\n",
    "2. At least 8GB RAM (16GB recommended)\n",
    "3. 10GB free disk space for model\n",
    "4. PostgreSQL installed"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step1",
   "metadata": {},
   "source": [
    "## STEP 1: Install Required Packages\n",
    "Run this cell first. It will take 5-10 minutes to install everything."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "install",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install all required packages\n",
    "!pip install --upgrade pip\n",
    "!pip install langchain langchain-community\n",
    "!pip install sentence-transformers  # For embeddings\n",
    "!pip install chromadb  # Vector database\n",
    "!pip install llama-cpp-python  # To run Llama locally\n",
    "!pip install psycopg2-binary  # PostgreSQL connector\n",
    "!pip install huggingface-hub  # To download models\n",
    "\n",
    "print(\"\\n✅ All packages installed successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step2",
   "metadata": {},
   "source": [
    "## STEP 2: Download Llama Model\n",
    "This will download a ~4GB model file. Takes 10-30 minutes depending on internet speed.\n",
    "We're using Llama-2-7B quantized version (smaller, faster)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "download_model",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "from huggingface_hub import hf_hub_download\n",
    "\n",
    "# Create models directory\n",
    "os.makedirs(\"models\", exist_ok=True)\n",
    "\n",
    "print(\"Downloading Llama model... This may take 10-30 minutes.\")\n",
    "print(\"Please be patient!\\n\")\n",
    "\n",
    "# Download the model\n",
    "model_path = hf_hub_download(\n",
    "    repo_id=\"TheBloke/Llama-2-7B-Chat-GGUF\",\n",
    "    filename=\"llama-2-7b-chat.Q4_K_M.gguf\",\n",
    "    local_dir=\"./models\"\n",
    ")\n",
    "\n",
    "print(f\"\\n✅ Model downloaded successfully!\")\n",
    "print(f\"Model location: {model_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step3",
   "metadata": {},
   "source": [
    "## STEP 3: Import Libraries and Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "imports",
   "metadata": {},
   "outputs": [],
   "source": [
    "import psycopg2\n",
    "from psycopg2 import sql\n",
    "from datetime import datetime, timedelta\n",
    "from langchain.embeddings import HuggingFaceEmbeddings\n",
    "from langchain.vectorstores import Chroma\n",
    "from langchain.text_splitter import RecursiveCharacterTextSplitter\n",
    "from langchain.llms import LlamaCpp\n",
    "from langchain.chains import ConversationalRetrievalChain\n",
    "from langchain.memory import ConversationBufferMemory\n",
    "from langchain.prompts import PromptTemplate\n",
    "from langchain.schema import Document\n",
    "from langchain.callbacks.manager import CallbackManager\n",
    "from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler\n",
    "\n",
    "print(\"✅ All libraries imported successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step4",
   "metadata": {},
   "source": [
    "## STEP 4: Database Configuration\n",
    "⚠️ IMPORTANT: Update these values with your PostgreSQL credentials!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "db_config",
   "metadata": {},
   "outputs": [],
   "source": [
    "# DATABASE CONFIGURATION - UPDATE THESE VALUES!\n",
    "DB_CONFIG = {\n",
    "    'dbname': 'medical_chatbot',     # Database name\n",
    "    'user': 'postgres',               # Your PostgreSQL username\n",
    "    'password': 'your_password',      # Your PostgreSQL password\n",
    "    'host': 'localhost',              # Usually localhost\n",
    "    'port': '5432'                    # Default PostgreSQL port\n",
    "}\n",
    "\n",
    "print(\"✅ Database configuration set!\")\n",
    "print(\"⚠️  Make sure to update the password and credentials!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step5",
   "metadata": {},
   "source": [
    "## STEP 5: Create Database and Tables\n",
    "This creates the database structure for storing appointments and patients."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "create_db",
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_database():\n",
    "    \"\"\"Create the database if it doesn't exist\"\"\"\n",
    "    try:\n",
    "        # Connect to PostgreSQL server (default 'postgres' database)\n",
    "        conn = psycopg2.connect(\n",
    "            dbname='postgres',\n",
    "            user=DB_CONFIG['user'],\n",
    "            password=DB_CONFIG['password'],\n",
    "            host=DB_CONFIG['host'],\n",
    "            port=DB_CONFIG['port']\n",
    "        )\n",
    "        conn.autocommit = True\n",
    "        cursor = conn.cursor()\n",
    "        \n",
    "        # Check if database exists\n",
    "        cursor.execute(\"SELECT 1 FROM pg_database WHERE datname = %s\", (DB_CONFIG['dbname'],))\n",
    "        exists = cursor.fetchone()\n",
    "        \n",
    "        if not exists:\n",
    "            cursor.execute(sql.SQL(\"CREATE DATABASE {}\").format(\n",
    "                sql.Identifier(DB_CONFIG['dbname'])\n",
    "            ))\n",
    "            print(f\"✅ Database '{DB_CONFIG['dbname']}' created successfully!\")\n",
    "        else:\n",
    "            print(f\"ℹ️  Database '{DB_CONFIG['dbname']}' already exists.\")\n",
    "        \n",
    "        cursor.close()\n",
    "        conn.close()\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Error creating database: {e}\")\n",
    "        return False\n",
    "    \n",
    "    return True\n",
    "\n",
    "def create_tables():\n",
    "    \"\"\"Create necessary tables for the chatbot\"\"\"\n",
    "    try:\n",
    "        conn = psycopg2.connect(**DB_CONFIG)\n",
    "        cursor = conn.cursor()\n",
    "        \n",
    "        # Create patients table\n",
    "        cursor.execute(\"\"\"\n",
    "            CREATE TABLE IF NOT EXISTS patients (\n",
    "                patient_id SERIAL PRIMARY KEY,\n",
    "                full_name VARCHAR(100) NOT NULL,\n",
    "                phone VARCHAR(20),\n",
    "                email VARCHAR(100),\n",
    "                date_registered TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n",
    "            )\n",
    "        \"\"\")\n",
    "        \n",
    "        # Create appointments table\n",
    "        cursor.execute(\"\"\"\n",
    "            CREATE TABLE IF NOT EXISTS appointments (\n",
    "                appointment_id SERIAL PRIMARY KEY,\n",
    "                patient_id INTEGER REFERENCES patients(patient_id),\n",
    "                appointment_date DATE NOT NULL,\n",
    "                appointment_time TIME NOT NULL,\n",
    "                reason TEXT,\n",
    "                status VARCHAR(20) DEFAULT 'scheduled',\n",
    "                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n",
    "                UNIQUE(appointment_date, appointment_time)\n",
    "            )\n",
    "        \"\"\")\n",
    "        \n",
    "        # Create chat_history table\n",
    "        cursor.execute(\"\"\"\n",
    "            CREATE TABLE IF NOT EXISTS chat_history (\n",
    "                chat_id SERIAL PRIMARY KEY,\n",
    "                patient_id INTEGER REFERENCES patients(patient_id),\n",
    "                user_message TEXT,\n",
    "                bot_response TEXT,\n",
    "                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n",
    "            )\n",
    "        \"\"\")\n",
    "        \n",
    "        conn.commit()\n",
    "        cursor.close()\n",
    "        conn.close()\n",
    "        \n",
    "        print(\"✅ All tables created successfully!\")\n",
    "        print(\"   - patients\")\n",
    "        print(\"   - appointments\")\n",
    "        print(\"   - chat_history\")\n",
    "        return True\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Error creating tables: {e}\")\n",
    "        return False\n",
    "\n",
    "# Execute database setup\n",
    "print(\"Setting up database...\\n\")\n",
    "if create_database():\n",
    "    create_tables()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step6",
   "metadata": {},
   "source": [
    "## STEP 6: Database Helper Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "db_functions",
   "metadata": {},
   "outputs": [],
   "source": [
    "class DatabaseManager:\n",
    "    \"\"\"Handles all database operations\"\"\"\n",
    "    \n",
    "    @staticmethod\n",
    "    def get_connection():\n",
    "        \"\"\"Get database connection\"\"\"\n",
    "        return psycopg2.connect(**DB_CONFIG)\n",
    "    \n",
    "    @staticmethod\n",
    "    def add_patient(full_name, phone=None, email=None):\n",
    "        \"\"\"Add a new patient or return existing patient ID\"\"\"\n",
    "        try:\n",
    "            conn = DatabaseManager.get_connection()\n",
    "            cursor = conn.cursor()\n",
    "            \n",
    "            # Check if patient exists\n",
    "            cursor.execute(\n",
    "                \"SELECT patient_id FROM patients WHERE full_name = %s\",\n",
    "                (full_name,)\n",
    "            )\n",
    "            result = cursor.fetchone()\n",
    "            \n",
    "            if result:\n",
    "                patient_id = result[0]\n",
    "            else:\n",
    "                # Insert new patient\n",
    "                cursor.execute(\n",
    "                    \"\"\"INSERT INTO patients (full_name, phone, email) \n",
    "                       VALUES (%s, %s, %s) RETURNING patient_id\"\"\",\n",
    "                    (full_name, phone, email)\n",
    "                )\n",
    "                patient_id = cursor.fetchone()[0]\n",
    "                conn.commit()\n",
    "            \n",
    "            cursor.close()\n",
    "            conn.close()\n",
    "            return patient_id\n",
    "            \n",
    "        except Exception as e:\n",
    "            print(f\"Error adding patient: {e}\")\n",
    "            return None\n",
    "    \n",
    "    @staticmethod\n",
    "    def book_appointment(patient_name, date, time, reason):\n",
    "        \"\"\"Book a new appointment\"\"\"\n",
    "        try:\n",
    "            conn = DatabaseManager.get_connection()\n",
    "            cursor = conn.cursor()\n",
    "            \n",
    "            # Get or create patient\n",
    "            patient_id = DatabaseManager.add_patient(patient_name)\n",
    "            \n",
    "            if not patient_id:\n",
    "                return \"Error: Could not create patient record.\"\n",
    "            \n",
    "            # Check if slot is available\n",
    "            cursor.execute(\n",
    "                \"\"\"SELECT appointment_id FROM appointments \n",
    "                   WHERE appointment_date = %s AND appointment_time = %s \n",
    "                   AND status = 'scheduled'\"\"\",\n",
    "                (date, time)\n",
    "            )\n",
    "            \n",
    "            if cursor.fetchone():\n",
    "                cursor.close()\n",
    "                conn.close()\n",
    "                return f\"Sorry, the slot at {time} on {date} is already booked. Please choose another time.\"\n",
    "            \n",
    "            # Book appointment\n",
    "            cursor.execute(\n",
    "                \"\"\"INSERT INTO appointments (patient_id, appointment_date, appointment_time, reason) \n",
    "                   VALUES (%s, %s, %s, %s) RETURNING appointment_id\"\"\",\n",
    "                (patient_id, date, time, reason)\n",
    "            )\n",
    "            \n",
    "            appointment_id = cursor.fetchone()[0]\n",
    "            conn.commit()\n",
    "            cursor.close()\n",
    "            conn.close()\n",
    "            \n",
    "            return f\"✅ Appointment booked successfully!\\nAppointment ID: {appointment_id}\\nDate: {date}\\nTime: {time}\\nPatient: {patient_name}\"\n",
    "            \n",
    "        except Exception as e:\n",
    "            return f\"Error booking appointment: {e}\"\n",
    "    \n",
    "    @staticmethod\n",
    "    def view_appointments(patient_name=None):\n",
    "        \"\"\"View appointments for a patient or all appointments\"\"\"\n",
    "        try:\n",
    "            conn = DatabaseManager.get_connection()\n",
    "            cursor = conn.cursor()\n",
    "            \n",
    "            if patient_name:\n",
    "                cursor.execute(\"\"\"\n",
    "                    SELECT a.appointment_id, p.full_name, a.appointment_date, \n",
    "                           a.appointment_time, a.reason, a.status\n",
    "                    FROM appointments a\n",
    "                    JOIN patients p ON a.patient_id = p.patient_id\n",
    "                    WHERE p.full_name = %s AND a.status = 'scheduled'\n",
    "                    ORDER BY a.appointment_date, a.appointment_time\n",
    "                \"\"\", (patient_name,))\n",
    "            else:\n",
    "                cursor.execute(\"\"\"\n",
    "                    SELECT a.appointment_id, p.full_name, a.appointment_date, \n",
    "                           a.appointment_time, a.reason, a.status\n",
    "                    FROM appointments a\n",
    "                    JOIN patients p ON a.patient_id = p.patient_id\n",
    "                    WHERE a.status = 'scheduled'\n",
    "                    ORDER BY a.appointment_date, a.appointment_time\n",
    "                \"\"\")\n",
    "            \n",
    "            appointments = cursor.fetchall()\n",
    "            cursor.close()\n",
    "            conn.close()\n",
    "            \n",
    "            if not appointments:\n",
    "                return \"No scheduled appointments found.\"\n",
    "            \n",
    "            result = \"\\n📅 Scheduled Appointments:\\n\" + \"=\"*50 + \"\\n\"\n",
    "            for apt in appointments:\n",
    "                result += f\"ID: {apt[0]} | Patient: {apt[1]} | Date: {apt[2]} | Time: {apt[3]} | Reason: {apt[4]}\\n\"\n",
    "            \n",
    "            return result\n",
    "            \n",
    "        except Exception as e:\n",
    "            return f\"Error viewing appointments: {e}\"\n",
    "    \n",
    "    @staticmethod\n",
    "    def cancel_appointment(appointment_id):\n",
    "        \"\"\"Cancel an appointment\"\"\"\n",
    "        try:\n",
    "            conn = DatabaseManager.get_connection()\n",
    "            cursor = conn.cursor()\n",
    "            \n",
    "            cursor.execute(\n",
    "                \"UPDATE appointments SET status = 'cancelled' WHERE appointment_id = %s RETURNING appointment_id\",\n",
    "                (appointment_id,)\n",
    "            )\n",
    "            \n",
    "            result = cursor.fetchone()\n",
    "            conn.commit()\n",
    "            cursor.close()\n",
    "            conn.close()\n",
    "            \n",
    "            if result:\n",
    "                return f\"✅ Appointment {appointment_id} has been cancelled successfully.\"\n",
    "            else:\n",
    "                return f\"Appointment {appointment_id} not found.\"\n",
    "                \n",
    "        except Exception as e:\n",
    "            return f\"Error cancelling appointment: {e}\"\n",
    "    \n",
    "    @staticmethod\n",
    "    def get_available_slots(date):\n",
    "        \"\"\"Get available time slots for a given date\"\"\"\n",
    "        try:\n",
    "            conn = DatabaseManager.get_connection()\n",
    "            cursor = conn.cursor()\n",
    "            \n",
    "            # Get booked times\n",
    "            cursor.execute(\n",
    "                \"\"\"SELECT appointment_time FROM appointments \n",
    "                   WHERE appointment_date = %s AND status = 'scheduled'\"\"\",\n",
    "                (date,)\n",
    "            )\n",
    "            \n",
    "            booked_times = [str(row[0]) for row in cursor.fetchall()]\n",
    "            cursor.close()\n",
    "            conn.close()\n",
    "            \n",
    "            # Define all possible slots\n",
    "            all_slots = ['09:00:00', '10:00:00', '11:00:00', '14:00:00', '15:00:00', '16:00:00']\n",
    "            available_slots = [slot for slot in all_slots if slot not in booked_times]\n",
    "            \n",
    "            if available_slots:\n",
    "                return \"Available slots for \" + date + \":\\n\" + \"\\n\".join([f\"  - {slot[:5]}\" for slot in available_slots])\n",
    "            else:\n",
    "                return f\"No available slots for {date}. Please choose another date.\"\n",
    "                \n",
    "        except Exception as e:\n",
    "            return f\"Error checking available slots: {e}\"\n",
    "    \n",
    "    @staticmethod\n",
    "    def save_chat_history(patient_name, user_message, bot_response):\n",
    "        \"\"\"Save chat conversation to database\"\"\"\n",
    "        try:\n",
    "            conn = DatabaseManager.get_connection()\n",
    "            cursor = conn.cursor()\n",
    "            \n",
    "            patient_id = DatabaseManager.add_patient(patient_name)\n",
    "            \n",
    "            cursor.execute(\n",
    "                \"\"\"INSERT INTO chat_history (patient_id, user_message, bot_response) \n",
    "                   VALUES (%s, %s, %s)\"\"\",\n",
    "                (patient_id, user_message, bot_response)\n",
    "            )\n",
    "            \n",
    "            conn.commit()\n",
    "            cursor.close()\n",
    "            conn.close()\n",
    "            \n",
    "        except Exception as e:\n",
    "            print(f\"Error saving chat history: {e}\")\n",
    "\n",
    "print(\"✅ Database Manager class created successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step7",
   "metadata": {},
   "source": [
    "## STEP 7: Medical Knowledge Base"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "medical_knowledge",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Expanded medical knowledge base\n",
    "medical_documents = [\n",
    "    \"\"\"Common Cold: A viral infection of the upper respiratory tract.\n",
    "    Symptoms: Runny nose, sore throat, cough, congestion, mild fever, sneezing, body aches.\n",
    "    Treatment: Rest, drink plenty of fluids, over-the-counter pain relievers like acetaminophen or ibuprofen.\n",
    "    Duration: Usually resolves in 7-10 days.\n",
    "    When to see a doctor: Fever above 101.3°F (38.5°C), symptoms lasting more than 10 days, difficulty breathing, severe headache.\"\"\",\n",
    "    \n",
    "    \"\"\"Headaches: Pain in the head or upper neck.\n",
    "    Types: Tension headaches (most common, feels like tight band), Migraines (severe throbbing pain, often one-sided, with nausea and light sensitivity), Cluster headaches (severe pain around one eye).\n",
    "    Treatment: Rest in a quiet dark room, stay hydrated, pain relievers, cold or warm compress.\n",
    "    When to see a doctor: Sudden severe headache, headache with fever or stiff neck, headache after head injury, changes in vision.\"\"\",\n",
    "    \n",
    "    \"\"\"Diabetes: A chronic condition affecting blood sugar regulation.\n",
    "    Type 1: Body doesn't produce insulin, usually diagnosed in children and young adults.\n",
    "    Type 2: Body doesn't use insulin properly, most common type, linked to obesity and lifestyle.\n",
    "    Symptoms: Increased thirst, frequent urination, extreme fatigue, blurred vision, slow-healing wounds, unexplained weight loss.\n",
    "    Management: Healthy diet, regular exercise, blood sugar monitoring, medication or insulin as prescribed.\n",
    "    Complications if untreated: Heart disease, kidney damage, nerve damage, vision problems.\"\"\",\n",
    "    \n",
    "    \"\"\"Hypertension (High Blood Pressure): Blood pressure consistently above normal levels.\n",
    "    Normal: Less than 120/80 mmHg. Elevated: 120-129/less than 80. High: 130/80 or higher.\n",
    "    Often has no symptoms, called the 'silent killer'.\n",
    "    Risk factors: Age, family history, obesity, lack of exercise, high salt diet, excessive alcohol, stress.\n",
    "    Management: DASH diet (low sodium, high fruits/vegetables), regular exercise, maintain healthy weight, limit alcohol, stress management, medication if prescribed.\n",
    "    Complications: Heart attack, stroke, kidney disease, vision loss.\"\"\",\n",
    "    \n",
    "    \"\"\"Fever: Elevated body temperature above 100.4°F (38°C).\n",
    "    Common causes: Infections (viral or bacterial), flu, COVID-19, urinary tract infections, pneumonia.\n",
    "    Treatment: Rest, drink plenty of fluids, acetaminophen or ibuprofen to reduce fever.\n",
    "    When to seek emergency care: Fever above 103°F (39.4°C), persistent fever for more than 3 days, severe headache, difficulty breathing, chest pain, persistent vomiting, confusion, seizures.\n",
    "    For children: See doctor if fever in infant under 3 months, or if child is listless or irritable.\"\"\",\n",
    "    \n",
    "    \"\"\"Asthma: A chronic respiratory condition causing airway inflammation.\n",
    "    Symptoms: Wheezing, shortness of breath, chest tightness, coughing (especially at night or early morning).\n",
    "    Triggers: Allergens (pollen, dust mites, pet dander), exercise, cold air, smoke, strong odors, respiratory infections.\n",
    "    Treatment: Quick-relief inhalers for acute symptoms, long-term control medications, avoiding triggers.\n",
    "    Emergency signs: Severe difficulty breathing, bluish lips or face, rapid worsening of symptoms.\"\"\",\n",
    "    \n",
    "    \"\"\"Allergies: Immune system reaction to normally harmless substances.\n",
    "    Common allergens: Pollen, dust mites, pet dander, certain foods (peanuts, shellfish, eggs), insect stings, medications.\n",
    "    Symptoms: Sneezing, runny nose, itchy eyes, skin rashes, in severe cases anaphylaxis.\n",
    "    Treatment: Avoid allergens, antihistamines, nasal corticosteroids, allergy shots (immunotherapy).\n",
    "    Anaphylaxis warning signs: Difficulty breathing, swelling of throat/tongue, rapid pulse, dizziness - requires immediate emergency care.\"\"\",\n",
    "    \n",
    "    \"\"\"Depression: A mental health disorder affecting mood and daily functioning.\n",
    "    Symptoms: Persistent sadness, loss of interest in activities, changes in appetite or sleep, fatigue, difficulty concentrating, feelings of worthlessness, thoughts of death or suicide.\n",
    "    Treatment: Psychotherapy (talk therapy), medications (antidepressants), lifestyle changes (exercise, healthy diet, social support).\n",
    "    Important: If you have thoughts of suicide, seek immediate help - call emergency services or a crisis hotline.\"\"\",\n",
    "    \n",
    "    \"\"\"Back Pain: Discomfort in the back, very common condition.\n",
    "    Causes: Muscle strain, poor posture, herniated disc, arthritis, osteoporosis.\n",
    "    Treatment: Rest (but not too much), ice/heat therapy, over-the-counter pain relievers, gentle stretching, physical therapy.\n",
    "    Prevention: Maintain good posture, exercise regularly, lift properly, maintain healthy weight.\n",
    "    See a doctor if: Pain lasts more than a few weeks, severe pain, pain radiating down legs, numbness or weakness, loss of bowel/bladder control.\"\"\"\n",
    "]\n",
    "\n",
    "print(\"✅ Medical knowledge base loaded with\", len(medical_documents), \"documents\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "step8",
   "metadata": {},
   "source": [
    "## STEP 8: Setup RAG System with Local Embeddings\n",
    "This uses FREE local embeddings (no API needed!)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "setup_rag",
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Setting up RAG system...\")\n",
    "print(\"This may take 2-3 minutes on first run (downloading embedding model)\\n\")\n",
    "\n",
    "# Convert to Document objects\n",
    "docs = [Document(page_content=doc) for doc in medical_documents]\n",
    "\n",
    "# Split documents\n",
    "text_splitter = RecursiveCharacterTextSplitter(\n",
    "    chunk_size=500,\n",
    "    chunk_overlap=50\n",
    ")\n",
    "splits = text_splitter.split_documents(docs)\n",
    "print(f\"Split into {len(splits)} chunks\")\n",
    "\n",
    "# Create embeddings using FREE local model\n",
    "embeddings = HuggingFaceEmbeddings(\n",
    "    model_name=\"sentence-transformers/all-MiniLM-L6-v2\",\n",
    "    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU\n",
    ")\n",
    "print(\"Embeddings model loaded\")\n",
    "\n",
    "# Create vector store\n",
    "vectorstore = Chroma.from_documents(\

SyntaxError: incomplete input (ipython-input-2183166806.py, line 596)